# KSSL
> Accessing and bundling [USDA/KSSL](https://www.nrcs.usda.gov/conservation-basics/natural-resource-concerns/soil/kellogg-soil-survey-laboratory-kssl) MIRS & NIRS dataset

The KSSL MIRS/NIRS database is only available on request to USDA/KSSL for now. The following utilities assume you have the database locally (both the Microsoft Access database and Bruker Optics MIRS and NIRS scans).

In [1]:
#| default_exp data.external.kssl

In [2]:
#| hide
from nbdev.showdoc import *
from nbdev.cli import *

%load_ext autoreload
%autoreload 2

In [3]:
#| export
from fastcore.basics import patch
from fastcore.xtras import mkdir
import pandas as pd
from pathlib import Path
import shutil
from shutil import rmtree
import glob
import re
from tqdm import tqdm
from typing import Dict, Callable

from spanda.utils import select_rows
from spanda.readers import read_opus, read_spectra, read_asd
from spanda.sig import interp

In [4]:
# src_dir = '../_data/kssl-202212'
src_dir = '../_data/kssl-202003'
dest_dir = '../_data/kssl-vnir'
#src_dir_mir_spectra = '/Users/franckalbinet/pro/data/MIR_Library'
src_dir_vnir_spectra = '/Users/franckalbinet/pro/data/VNIR_Spectral_Library'

In [5]:
#| export
class KSSL:
    def __init__(self, 
                 src_dir:str, # KSSL directory 
                ):
        self.src_dir = Path(src_dir)

In [6]:
#| export
@patch
def to_csv(self:KSSL,
           dest_dir:str, # Destination directory
          ):
    "Convert KSSL Microsoft Access DB tables to individual `.csv` files" 
    pass

In [7]:
#| export
@patch
def get_sample(self:KSSL):
    """Return KSSL DB `sample` table as `pd.DataFrame`
    
        - Only `smp_id` > 1000 are selected
    """ 
    return pd.read_csv(self.src_dir / 'sample.csv', low_memory=False) \
        .pipe(select_rows, {'smp_id': lambda d: d > 1000}) \
        .loc[:, ['smp_id', 'lay_id']]

In [8]:
#|eval: false
kssl = KSSL(src_dir)
df = kssl.get_sample(); df.head()

,smp_id,lay_id
1,25111,123
2,25147,160
3,25590,271
4,25597,278
5,25604,285


In [9]:
#| export
@patch
def get_layer(self:KSSL) -> pd.DataFrame: # Processed DataFrame
    "Return KSSL DB `layer` table as `pd.DataFrame`" 
    return pd.read_csv(self.src_dir / 'layer.csv', low_memory=False) \
        .loc[:, ['lay_id', 'lims_pedon_id', 'lims_site_id', 'lay_depth_to_top']] \
        .dropna() \
        .astype({'lims_pedon_id': 'int32', 'lims_site_id': 'int32'})

In [10]:
#|eval: false
kssl = KSSL(src_dir)
df = kssl.get_layer(); df.head()

,lay_id,lims_pedon_id,lims_site_id,lay_depth_to_top
0,587,120,121,28.0
1,596,121,122,81.0
2,621,125,126,61.0
3,630,126,127,76.0
4,689,138,139,3.0


In [11]:
#| export
@patch
def get_layer_analyte(self:KSSL):
    """Return KSSL DB `layer_analyte` table as `pd.DataFrame`
        
        - Only `master_prep_id` relevant to MIRS analysis selected: `[18, 19, 27, 28]`
        - `calc_value` are by default `str` as possibly containing values such as (`slight`, `1:2`, ...). 
          Only numeric ones are selected.
    """
    value_pred = lambda d: (re.search(r'[a-zA-Z]|:|\s', str(d)) is None # Should not contains string-like char
                             and float(d) > 0) # only strictly positive values
    
    return pd.read_csv(self.src_dir / 'layer_analyte.csv', low_memory=False) \
        .dropna(subset=['analyte_id', 'calc_value']) \
        .pipe(select_rows, {
            'master_prep_id': lambda d: d in [18, 19, 27, 28],
            'calc_value': value_pred}) \
        .loc[:, ['lay_id', 'analyte_id', 'calc_value']] \
        .astype({'calc_value': float})

In [12]:
#|eval: false
kssl = KSSL(src_dir)
df = kssl.get_layer_analyte(); df.head()

,lay_id,analyte_id,calc_value
26,110524,336,12.685978
27,110524,334,38.140311
29,110524,337,38.140311
30,110524,338,0.656713
31,110524,339,4.535427


In [13]:
#| export
@patch
def get_mir_mas(self:KSSL):
    "Return KSSL DB `mir_scan_mas_data.csv` table as `pd.DataFrame`"
    return pd.read_csv(self.src_dir / 'mir_scan_mas_data.csv', low_memory=False) \
        .loc[:, ['smp_id', 'mir_scan_mas_id']]

In [14]:
#|eval: false
kssl = KSSL(src_dir)
df = kssl.get_mir_mas(); df.head()

,smp_id,mir_scan_mas_id
0,128202,4115
1,128208,4122
2,128230,4145
3,132827,4179
4,132829,4181


In [15]:
#| export
@patch
def get_mir_det(self:KSSL):
    """Return KSSL DB `vnir_scan_mas_data` table as `pd.DataFrame`
    
        - Only `scan_path_name` containing valid substring `['XN', 'XS']

    """ 
    return pd.read_csv(self.src_dir / 'mir_scan_det_data.csv', low_memory=False) \
        .dropna(subset=['scan_path_name', 'mir_scan_mas_id']) \
        .loc[:, ['mir_scan_mas_id', 'scan_path_name']] \
        .pipe(select_rows, {
            'scan_path_name': lambda d: re.search(r'X.', str(d))[0] in ['XN', 'XS']})

In [16]:
#|eval: false
kssl = KSSL(src_dir)
df = kssl.get_mir_det(); df.head()

,mir_scan_mas_id,scan_path_name
0,14,137437XS01.0
1,14,137437XS02.0
2,14,137437XS03.0
3,14,137437XS04.0
4,16,146625XS01.0


In [17]:
#| export
@patch
def get_vnir_mas(self:KSSL):
    "Return KSSL DB `nir_scan_mas_data.csv` table as `pd.DataFrame`"
    return pd.read_csv(self.src_dir / 'vnir_scan_mas_data.csv', low_memory=False) \
        .loc[:, ['smp_id', 'vnir_scan_mas_id']]

In [18]:
#|eval: false
kssl = KSSL(src_dir)
df = kssl.get_vnir_mas(); df.head()

,smp_id,vnir_scan_mas_id
0,146787,6134
1,146823,6170
2,146974,6313
3,146981,6320
4,146988,6327


In [19]:
#| export
@patch
def get_vnir_det(self:KSSL):
    """Return KSSL DB `nir_scan_det_data.csv` table as `pd.DataFrame`
    
        #- Only `scan_path_name` containing valid substring `['XN', 'XS']

    """ 
    return pd.read_csv(self.src_dir / 'vnir_scan_det_data.csv', low_memory=False) \
        .dropna(subset=['scan_path_name', 'vnir_scan_mas_id']) \
        .loc[:, ['vnir_scan_mas_id', 'scan_path_name']]

In [20]:
#|eval: false
kssl = KSSL(src_dir)
df = kssl.get_vnir_det(); df.head()

,vnir_scan_mas_id,scan_path_name
0,1,136460CF01.asd
1,1,136460CF02.asd
2,1,136460CF03.asd
3,2,136461CF01.asd
4,2,136461CF02.asd


In [21]:
#| export
@patch
def get_analyte_lut(self:KSSL):
    "Return KSSL DB `analyte` lookup table as `pd.DataFrame`"""
    df = pd.read_csv(self.src_dir / 'analyte.csv') \
        .loc[:, ['analyte_id', 'analyte_name', 'analyte_abbrev', 'uom_abbrev']]
    return df

In [22]:
#|eval: false
kssl = KSSL(src_dir)
df = kssl.get_analyte_lut(); df.head()

,analyte_id,analyte_name,analyte_abbrev,uom_abbrev
0,1,"Aggregate Stability, 0.5-2mm Aggregates",aggstb,% wt
1,2,"Liquid Limit, Atterberg",abgll,% H2O
2,3,"Plastic Limit, Atterberg",abgpl,% H2O
3,4,"Bulk Density, <2mm Fraction, 1/3 Bar",db_13b,g/cc
4,5,"Bulk Density, <2mm Fraction, Ovendry",db_od,g/cc


In [30]:
#|eval: false
df[df['analyte_name'].str.contains('pH')]

,analyte_id,analyte_name,analyte_abbrev,uom_abbrev
67,82,"CEC, NH4OAc, pH 7.0",cec_nh4,cmol(+)/kg
72,88,"Acidity, BaCl2-TEA Extractable, pH 8.2",acid_tea,cmol(+)/kg
122,795,"Acidity, BaCl2-TEA Extractable, pH 8.2, centri...",acid_teac,cmol(+)/kg
273,265,"pH, 1:1 Soil-KCl Suspension",ph_kcl,(NA)
274,266,"pH, 1:50 Soil-NaFl Suspension",ph_naf,(NA)
275,267,"pH, Oxidized (sulfidic)",ph_ox,(NA)
276,268,"pH, 1:1 Soil-Water Suspension",ph_h2o,(NA)
305,477,"pH, 0.01M CaCl2, Histosol",ph_hist,(NA)
309,481,"pH, 1:2 Soil-CaCl2 Suspension",ph_cacl2,(NA)
310,482,"pH, Saturated Paste",ph_sp,(NA)


In [ ]:
#| export
@patch
def get_taxonomy_lut(self:KSSL):
    pass

In [ ]:
#| export
@patch
def get_counts(self:KSSL):
    pass

In [ ]:
#| export
@patch
def join_mir(self:KSSL,
              analytes:list[int]=[725, 336], # Analytes of interest
             ):
    "Return master MIR join to be exported as `pd.DataFrame`"
    lay_smp = pd.merge(self.get_layer(),
                       self.get_sample(), 
                       on='lay_id', how='inner').sort_values(by='lay_id')

    lay_smp_mirmas = pd.merge(lay_smp, 
                               self.get_mir_mas(), 
                               on='smp_id', how='inner')
    
    layer_analyte_of_interest = self.get_layer_analyte() \
                                    .pipe(select_rows, 
                                          {'analyte_id': lambda d: d in analytes})
    
    lay_anal_wide = pd.pivot_table(layer_analyte_of_interest,
                                   values='calc_value',
                                   index=['lay_id'], columns=['analyte_id'])
    
    lay_smp_mir_mas_det = pd.merge(lay_smp_mirmas,
                                    self.get_mir_det(),
                                    on='mir_scan_mas_id', how='inner')
    
    return pd.merge(lay_smp_mir_mas_det,
                    lay_anal_wide, on='lay_id', how='left').dropna(subset=analytes)

In [ ]:
#|eval: false
kssl = KSSL(src_dir)
df = kssl.join_mir(analytes=[725, 723, 334]); df

,lay_id,lims_pedon_id,lims_site_id,lay_depth_to_top,smp_id,mir_scan_mas_id,scan_path_name,334,723,725
372,286,67,68,25.0,25605,47176,25605XS01.0,8.654031,9.792902,0.212940
373,286,67,68,25.0,25605,47176,25605XS02.0,8.654031,9.792902,0.212940
374,286,67,68,25.0,25605,47176,25605XS03.0,8.654031,9.792902,0.212940
375,286,67,68,25.0,25605,47176,25605XS04.0,8.654031,9.792902,0.212940
376,287,68,69,0.0,25606,47177,25606XS01.0,11.670895,32.497067,0.547895
...,...,...,...,...,...,...,...,...,...,...
314663,198661,40205,33851,67.0,291827,87466,291827XS04.0,39.759334,20.297427,0.483400
314664,198662,40205,33851,120.0,291828,87467,291828XS01.0,18.725758,7.844801,0.257538
314665,198662,40205,33851,120.0,291828,87467,291828XS02.0,18.725758,7.844801,0.257538
314666,198662,40205,33851,120.0,291828,87467,291828XS03.0,18.725758,7.844801,0.257538


In [ ]:
#| export
@patch
def join_vnir(self:KSSL,
              analytes:list[int]=[725, 336], # Analytes of interest
             ):
    "Return master NIR join to be exported as `pd.DataFrame`"
    lay_smp = pd.merge(self.get_layer(),
                       self.get_sample(), 
                       on='lay_id', how='inner').sort_values(by='lay_id')

    lay_smp_vnirsmas = pd.merge(lay_smp, 
                               self.get_vnir_mas(), 
                               on='smp_id', how='inner')
    
    layer_analyte_of_interest = self.get_layer_analyte() \
                                    .pipe(select_rows, 
                                          {'analyte_id': lambda d: d in analytes})
    
    lay_anal_wide = pd.pivot_table(layer_analyte_of_interest,
                                   values='calc_value',
                                   index=['lay_id'], columns=['analyte_id'])
    
    lay_smp_vnir_mas_det = pd.merge(lay_smp_vnirsmas,
                                    self.get_vnir_det(),
                                    on='vnir_scan_mas_id', how='inner')
    
    return pd.merge(lay_smp_vnir_mas_det,
                    lay_anal_wide, on='lay_id', how='left').dropna(subset=analytes)

In [ ]:
#|eval: false
kssl = KSSL(src_dir)
df = kssl.join_vnir(analytes=[725, 723, 334]); df

,lay_id,lims_pedon_id,lims_site_id,lay_depth_to_top,smp_id,vnir_scan_mas_id,scan_path_name,334,723,725
0,32383,9079,9122,0.0,87230,18909,87230MD01.asd,8.684919,6.410900,0.794636
1,32384,9079,9122,2.0,87231,18910,87231MD01.asd,9.171114,6.096263,0.749588
2,32385,9079,9122,7.0,87232,18911,87232MD01.asd,9.838435,5.741733,1.035568
3,32386,9080,9123,0.0,87233,18912,87233MD01.asd,4.072871,4.046288,0.448437
4,32387,9080,9123,2.0,87234,18913,87234MD01.asd,6.765260,4.563653,0.407192
...,...,...,...,...,...,...,...,...,...,...
67212,171020,34485,28323,103.0,238096,59729,238096MD01.asd,2.933274,4.080152,0.125758
67213,171021,34486,28324,0.0,238097,59730,238097MD01.asd,0.080398,0.316598,0.025112
67215,171023,34487,28325,16.0,238099,59732,238099MD01.asd,0.407463,1.638778,0.024870
67217,171025,34487,28325,36.0,238101,59734,238101MD01.asd,3.897087,5.304709,0.048181


In [ ]:
#| export
class SpectrumFinder:
    def __init__(self, src, pattern='.csv', lut=None):
        self.src = src
        self.pattern = pattern
        self.lut = self._create_lut() if lut is None else lut
        
    def _sanitize(self, fname):
        return Path(fname).stem
    
    def _create_lut(self):
        lut = {}
        for p in glob.iglob(self.src + '/*/*' + self.pattern):
            fname = Path(p)
            lut[fname.stem] = fname.parents[0].name
        return lut
    
    def find(self, fname):
        fname = self._sanitize(fname)
        try:
            parent_dir = self.lut[fname]
        except KeyError:
            return None
        return Path(self.src) / parent_dir / (fname + self.pattern)

In [ ]:
#|eval: false
src_dir_spectra = '/Users/franckalbinet/pro/data/VNIR_Spectral_Library'
specFinder = SpectrumFinder(src_dir_spectra, pattern='.asd', lut=None)

In [ ]:
#|eval: false
specFinder.lut['175590MD01']

'C2012USOH059'

In [ ]:
#|eval: false
specFinder.find('221114MD01.asd')

Path('/Users/franckalbinet/pro/data/VNIR_Spectral_Library/S2013USNL095/221114MD01.asd')

In [ ]:
#| export
is_spectrum_valid = lambda x: np.std(x) != 0

In [ ]:
#| export
@patch
def bundle_vnir(self:KSSL,
           src_dir_spectra:str, # Directory containing infrared spectra
           dest_dir:str, # Destination directory
           reader:Callable=read_spectra, # kssl spectra reader ex: (read_opus, read_spectra, ...)
           analytes:list[int]=[725, 723, 334], # Analytes of interest
           lut:dict=None, # spectrum name -> path lookup table
           compressed=True, # True if folder should be compressed
          ):
    """Bundle KSSL VNIR dataset as follows:

        - /dest_dir/
        -          /layer_id/
        -                    spectra_id1.csv
        -                    spectra_id2.csv
        -                    spectra_idx.csv
        -                    target.csv
    """
    dest_dir = Path(dest_dir)        
    mkdir(dest_dir, overwrite=True)
    
    df = self.join_vnir(analytes=analytes)
    
    # iterate and create tree structure
    #n = 0
    specFinder = SpectrumFinder(src_dir_spectra, pattern='.asd', lut=lut)
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        path = dest_dir / str(row['lay_id'])
        if not path.exists():
            path.mkdir()
            target = row[analytes]
            target.index.name = 'analyte'
            target.name = 'value'
            target.to_csv(path/'target.csv')

        # read, process and bundle spectra  
        fname = specFinder.find(row['scan_path_name'])
        if fname:
            absorbance, wn = reader(fname)
            if is_spectrum_valid(absorbance):            
                spectrum = pd.Series(data=absorbance, index=wn, name='absorbance')
                spectrum.index.name = 'wavelength'
                out_fname = str(path / Path(row['scan_path_name']).stem) + '.csv'
                spectrum.to_csv(out_fname)
        else:
            pass
        #n += 1
        #if n > 10: break

In [ ]:
#|eval: false
src_dir_vnir_spectra = '/Users/franckalbinet/pro/data/VNIR_Spectral_Library'
dest_dir = '../_data/kssl-vnir'

kssl = KSSL(src_dir)
df = kssl.bundle_vnir(src_dir_vnir_spectra, 
                      dest_dir,
                      reader=read_asd,
                      analytes=[725, 723, 334])

100%|█████████████████████████████████████████████████████████| 30618/30618 [02:35<00:00, 196.93it/s]


In [ ]:
#| export
@patch
def bundle_mir(self:KSSL,
           src_dir_spectra:str, # Directory containing infrared spectra
           dest_dir:str, # Destination directory
           reader:Callable=read_spectra, # kssl spectra reader ex: (read_opus, read_spectra, ...)
           analytes:list[int]=[725, 723, 334], # Analytes of interest
           lut:dict=None, # spectrum name -> path lookup table
           compressed=True, # True if folder should be compressed
          ):
    """Bundle KSSL dataset as follows:

        - /dest_dir/
        -          /layer_id/
        -                    target.csv
    """
    dest_dir = Path(dest_dir)        
    mkdir(dest_dir, overwrite=True)
    
    df = self.join_mir(analytes=analytes)
    
    # iterate and create tree structure
    n = 0
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        path = dest_dir / str(row['lay_id'])
        if not path.exists():
            path.mkdir()
            target = row[analytes]
            target.index.name = 'analyte'
            target.name = 'value'
            target.to_csv(path/'target.csv')

        # read, process and bundle spectra
        #path_spectra = row['scan_path_name'].split('.')[0] + '.csv'
        #fnames = glob.glob(src_dir_spectra + '/*/' + path_spectra, 
        #                     recursive=True)
        
        specFinder = SpectrumFinder(src_dir_spectra, lut=lut)
        fname = specFinder.find(row['scan_path_name'])
        if fname:
            absorbance, wn = interp(reader(fname))
            spectrum = pd.Series(data=absorbance, index=wn, name='absorbance')
            spectrum.index.name = 'wavenumber'
            #out_fname = str(path / Path(row['scan_path_name']).stem) + '.csv'
            #out_fname = str(path / Path(row['scan_path_name']).stem) + '.csv'
            print(path, ',', fname)
            #spectrum.to_csv(out_fname)
        else:
            pass
            #print('No spectra in folder: ', path, ' .Deleting it ...')
            #rmtree(path)
        n += 1
        if n > 10: break

In [ ]:
#src_dir = '../_data/kssl-202212'
#src_dir_spectra = '/Users/franckalbinet/pro/data/MIR_Library'
#src_dir_spectra = '/Volumes/BACKUP/kssl-202212/MIR_Library'
#dest_dir = '../_data/kssl-mirs-v2'

'/Users/franckalbinet/pro/data/VNIR_Spectral_Library'

In [ ]:
#|eval: false
kssl = KSSL(src_dir)
df = kssl.bundle_mir(src_dir_spectra, 
                 dest_dir,
                 lut=lut,
                 analytes=[725, 723, 334])

Reading and joining MIRS ...


  0%|                                                                  | 0/177597 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../_data/kssl-mirs-v2/R2001USMI077/25605XS01.csv'